###### References: 
- https://docs.python.org/3/library/functions.html   
- Fluent Python by Luciano Ramalho. Chapter 7: Function Decorators and Closures

# Decorators
Function decorators let us "mark" functions in the source codde to enhance their behaviour in some way.

It is a callable that takes another function as argument.

    @decorate
    def target():
        print('running  target()')

is same as:

    

In [1]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

In [2]:
@deco
def target():
    print('running target()')

In [3]:
target()

running inner()


In [4]:
target

<function __main__.deco.<locals>.inner()>

##  When  Python  Executes  Decorators
Usually at `import` time

In [5]:
%run registration.py

running register(<function f1 at 0x107489830>)
running register(<function f2 at 0x107489a70>)
running main()
registry -> [<function f1 at 0x107489830>, <function f2 at 0x107489a70>]
running f1()
running f2()
running f3()


In [6]:
import registration

running register(<function f1 at 0x10749c560>)
running register(<function f2 at 0x10749c830>)


In [7]:
registration.registry

[<function registration.f1()>, <function registration.f2()>]

##  Decorator-Enhanced Strategy Pattern
With a registration decorator

In [8]:
%run promotions.py

In [9]:
promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]

## Variable Scope Rules

In [10]:
def f1(a):
    print(a)
    print(b)

In [11]:
f1(3)

3


NameError: name 'b' is not defined

In [12]:
b = 6
f1(3)

3
6


In [13]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [14]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

f3(3)

3
6


In [15]:
b

9

# Closures
are functions with an extended scope that encompasses nonglobal variables referenced in the bodu of the function but no defined there.

eg. An averager that keeps track of a running average:

In [16]:
class Averager():

    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [17]:
avg = Averager()
avg(10)

10.0

In [18]:
avg(11)

10.5

In [19]:
avg(12)

11.0

In [20]:
def make_averager():

    # closure
    series = []
    def averager(new_value):
        series.append(new_value)  # `series` is a free  variable
        total = sum(series)
        return total/len(series)

    return averager

In [21]:
avg = make_averager()
avg(10)

10.0

In [22]:
avg(11)

10.5

In [23]:
avg(12)

11.0

####  `series` is a free variable. i.e. a variable not bound in the local scope.
![closure](closure.png)

In [24]:
avg.__code__.co_varnames

('new_value', 'total')

In [25]:
avg.__code__.co_freevars

('series',)

In [26]:
avg.__closure__

(<cell at 0x1075403d0: list object at 0x107503690>,)

In [27]:
avg.__closure__[0].cell_contents

[10, 11, 12]

##  The  nonlocal Declaration

In [28]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value 
        return total / count

    return averager

In [29]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [30]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value 
        return total / count

    return averager

In [31]:
avg = make_averager()
avg(10)

10.0

## Implementing a Simple Decorator

In [32]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.time()
        result = func(*args) # encompasses the func free variable
        elapsed = time.time() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked # return inner  function to replace decorated function

In [33]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

In [34]:
print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling factorial(6)')
print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12741208s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000095s] factorial(1) -> 1
[0.00005031s] factorial(2) -> 2
[0.00008297s] factorial(3) -> 6
[0.00092292s] factorial(4) -> 24
[0.00102687s] factorial(5) -> 120
[0.00109887s] factorial(6) -> 720
6! = 720


In [35]:
factorial.__name__

'clocked'

### `functools.wraps` to  handle keyword arguments

In [36]:
import functools

In [37]:
def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs) # include keyword arguments
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_str = ', '.join(repr(arg) for arg in args)
        if kwargs:
            pairs = ['%s=%r' %  (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(','.join(pairs))
        arg_str = ','.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [38]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling factorial(6)')
print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12761712s] snooze() -> None
**************************************** Calling factorial(6)
[0.00000095s] factorial() -> 1
[0.00057817s] factorial() -> 2
[0.00063181s] factorial() -> 6
[0.00070095s] factorial() -> 24
[0.00075126s] factorial() -> 120
[0.00079727s] factorial() -> 720
6! = 720


# Decorators in the Standard Library

## Memoization with `functools.lru_cache`
Memoization: an optimization technique that works by saving the results of previous invocations of an expensive function. 

In [39]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [40]:
print(fibonacci(6))

[0.00000000s] fibonacci() -> 0
[0.00000095s] fibonacci() -> 1
[0.00023007s] fibonacci() -> 1
[0.00000095s] fibonacci() -> 1
[0.00000000s] fibonacci() -> 0
[0.00000000s] fibonacci() -> 1
[0.00007105s] fibonacci() -> 1
[0.00014210s] fibonacci() -> 2
[0.00044417s] fibonacci() -> 3
[0.00000119s] fibonacci() -> 1
[0.00000000s] fibonacci() -> 0
[0.00000000s] fibonacci() -> 1
[0.00007010s] fibonacci() -> 1
[0.00014114s] fibonacci() -> 2
[0.00000095s] fibonacci() -> 0
[0.00000095s] fibonacci() -> 1
[0.00006986s] fibonacci() -> 1
[0.00000000s] fibonacci() -> 1
[0.00000095s] fibonacci() -> 0
[0.00000000s] fibonacci() -> 1
[0.00007296s] fibonacci() -> 1
[0.00014281s] fibonacci() -> 2
[0.00028396s] fibonacci() -> 3
[0.00066400s] fibonacci() -> 5
[0.00118423s] fibonacci() -> 8
8


In [41]:
@functools.lru_cache() # invoked as a regular function
@clock  # stacked decorators; 
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [42]:
print(fibonacci(6))

[0.00000119s] fibonacci() -> 0
[0.00000215s] fibonacci() -> 1
[0.00027180s] fibonacci() -> 1
[0.00000095s] fibonacci() -> 2
[0.00034618s] fibonacci() -> 3
[0.00000119s] fibonacci() -> 5
[0.00041699s] fibonacci() -> 8
8


## Generic Functions with Single Dispatch

In [43]:
import html

In [44]:
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

In [45]:
htmlize({1, 2, 3})

'<pre>{1, 2, 3}</pre>'

In [46]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [47]:
htmlize('Heimlich & Co.\n- a game')

'<pre>&#x27;Heimlich &amp; Co.\\n- a game&#x27;</pre>'

In [48]:
htmlize(42)

'<pre>42</pre>'

In [49]:
print(htmlize(['alpha', 66, {3, 2, 1}]))

<pre>[&#x27;alpha&#x27;, 66, {1, 2, 3}]</pre>


In [50]:
from functools import singledispatch
from collections import abc
import numbers

In [51]:
@singledispatch  # marks the base function that handles the object type
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)  # each specialised function is decorated
def _(text):            # the name of the function is irrelevant
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral)  # for each addditional type, register a new function
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)  # can stack several registers to support different types in same function
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

In [52]:
htmlize({1, 2, 3})

'<pre>{1, 2, 3}</pre>'

In [53]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [54]:
htmlize('Heimlich & Co.\n- a game')

'<p>Heimlich &amp; Co.<br>\n- a game</p>'

In [55]:
htmlize(42)

'<pre>42 (0x2a)</pre>'

In [56]:
print(htmlize(['alpha', 66, {3, 2, 1}]))

<ul>
<li><p>alpha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>


## Stack decorators
    @d1
    @d2
    def f():
        print('f')
        
is the same as:

    def f():
        print('f)
        
    f = d1(d2(f))

## Parameterized Decorators
To make it easy to enable or disable the function registration:

In [57]:
registry = set()  # type(set) to make adding and removing faster

def register(active=True):  # takes optional keyword argument
    def decorate(func):  # inner function is the decorator: it takes a function as an  argument
        print('running register(active=%s)->decorate(%s)'
              % (active, func))
        if active:   
            registry.add(func)
        else:
            registry.discard(func)

        return func
    return decorate  # register is our decorator factory

@register(active=False)  # invoked as a function, with the desired parameters
def f1():
    print('running f1()')

@register() 
def f2():
    print('running f2()')

def f3():
    print('running f3()')

running register(active=False)->decorate(<function f1 at 0x107468290>)
running register(active=True)->decorate(<function f2 at 0x10749c8c0>)


In [58]:
registry

{<function __main__.f2()>}

In [59]:
register()(f3)

running register(active=True)->decorate(<function f3 at 0x107468ef0>)


<function __main__.f3()>

In [60]:
registry

{<function __main__.f2()>, <function __main__.f3()>}

In [61]:
register(active=False)(f2)

running register(active=False)->decorate(<function f2 at 0x10749c8c0>)


<function __main__.f2()>

In [62]:
registry

{<function __main__.f3()>}

## The Parameterized Clock Decorator

In [63]:
DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):  # parameterized decorator factory
    def decorate(func):      # the actual decorator
        def clocked(*_args): # wraps the decorated function
            t0 = time.time()
            _result = func(*_args)  # actual result of the decorated function
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)  # _args holds actual arguments; args is used for display
            result = repr(_result)
            print(fmt.format(**locals()))  # allow any local variable to be referenced
            return _result  
        return clocked  
    return decorate 

In [64]:
@clock()
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

[0.12695098s] snooze(0.123) -> None
[0.12431598s] snooze(0.123) -> None
[0.12584686s] snooze(0.123) -> None


In [65]:
@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze: 0.12625885009765625s
snooze: 0.1282670497894287s
snooze: 0.12530875205993652s


In [66]:
@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.123s
snooze(0.123) dt=0.127s
snooze(0.123) dt=0.128s
